<h1>1-Lidando com valores perdidos</h1>

Limpeza da dedos é uma parte da ciência de dados, porém pode ser profundamente frustante. Qual o motivo de alguns dados estarem distorcidos? O que você deveria fazer com esses valores faltantes? Qual o motivo os seus dados não estarem formatados corretamente? Como limpar reapidamente seus dados inconsistentes? Nesta seção vamos responder essas perguntas.

Nesta lição o leitor aprenderar a lidar com as mais comuns técnicas de limpeza de dados, para que você possa analizar seus dados rapidamente.

<h1> Dê uma primeira olhada nos seus dados </h1>

A primeira coisa que você precisa fazer é carregar as bibliotecas e os dados que irá ser utilizado.

Para essa demonstração vamos utilizar dados do futebol americano.

<h2> Importante!! </h2>

Baixe os arquivos deste tutorial nos links:

1 - https://www.kaggle.com/aparnashastry/building-permit-applications-data

2 - https://www.kaggle.com/aparnashastry/building-permit-applications-data

In [2]:
#Bibliotecas que vamos utilizar
import pandas as pd
import numpy as np

#Lendo os dados
nfl = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")

#Setando uma semente para o que o leitor tenha mesmo resultado
np.random.seed(0)

A primeira coisa a fazer quando você pega um novo dataset é olhar parte dele. Fazer isso , permite que você veja que tudo foi lido corretamente e dá uma idéia de o que está acontecendo com seus dados. Neste caso, vamos ver se há algum valor faltante, no qual séra representado como `NaN` ou `None`

In [3]:
# Olharemos as primeiras 5 linhas do DataFrame
# Podemos ver valores faltantes.

nfl.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


<h1> Quantos valores faltantes nós temos? </h1>

Vamos ver quantos valores faltantes existe em cada coluna:

In [4]:
missing_values_count = nfl.isnull().sum()

#Vamos olhar nas primeiras 10 colunas 
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

Parece muitos! Pode nos ajudar ver a porcentagem de valores faltantes do nosso dataset.

In [6]:
total_celulas = np.product(nfl.shape)
total_missing = missing_values_count.sum()

porcentagem_faltante = (total_missing/total_celulas)*100
print(porcentagem_faltante)

24.87214126835169


Minha nossa, quase 1/4 do nosso dataset está vazio. No próximo passo, nós vamos olhar mais de perto para algumas colunas com dados vazios e tentar entender o que está acontecendo.

<h1> Descobrindo por que os dados estão vazios</h1>

Neste ponto entraremos em uma área de ciência de dados que chama `intuição de dados`, no qual eu quero dizer "Vamos olhar realmente para os dados e tentar entender por que está assim e como isso afeta nossa análise". Pode ser uma parte frustante, especialmente se você é novo na área e não tem muita experiência. Para lidar com dados vazios, você precisará da sua intuição para descobrir o porque desse fenômeno.
A primeira pergunta que você pode fazer é:

<b> Este valor está vazio por conta que não foi gravado ou por conta de não existir? </b>

Se o valor está vazio por conta que não existe(por exemplo, o peso da criança mais velha de alguém que nem tem criança) então não faz sentido tentar prever o valor vazio. Este valor, porvavelmente, nós queremos manter como `NaN`. Em contrapartida , se o valor está vazio por conta que não foi gravado, então nós podemos tentar advinhar seu valor baseado nos valores restantes da coluna. Isso se chama <b>imputation</b>, e vamos aprender já já.

In [8]:
#Ohando para os valores vazios das 10 primeiras colunas
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

Olhando a documentação, eu posso ver uma coluna que tem a informação do número de segundos restantes no jogo quando a jogada foi feita. Isto significa que esses valores provavelmente não foram gravados, ao invés de não existirem. Então, faia sentido para nós tentar e advinhar o que estes dados deveriam ser ao invés de simplesmente deixa-los vazios.

Em contrapartida, existem outras colunas, como "PenalizedTeam" que também possui muitos valores vazios. Neste caso, talvez, os campos vazios é devidos ao fato de termos ou não penalidades então não faz sentido dizer qual time foi penalizado. Para esta coluna, faz muito sentido deixar essa coluna com os valores vazios ou adicionar um terceiro valor que substitua os NA´s.

<b> Dica:</b> Este é um ótimo lugar para ler sobre a documentação do dataset se você ainda não leu. Se você está trabalhando com um dataset que você pegou de outa pessoa, você pode perguntar sobre as informações do mesmo. 

Se você estiver fazendo uma análise de dados muito cuidadosa, este é o ponto em que você examinaria cada coluna individualmente para descobrir a melhor estratégia para preencher esses valores ausentes. No restante deste estudo, cobriremos algumas técnicas "rápidas e sujas" que podem ajudá-lo com valores ausentes, mas provavelmente também acabarão removendo algumas informações úteis ou adicionando algum ruído aos seus dados.

<h1> Tirando valores vazios </h1>

Se você estiver com pressa ou não tem motivo se saber o motivo do por que existe valores vazios, uma opção que você tem é simplesmente qualquer linha ou coluna que contenha valores vazios( não recomendo esse approch para projetos importantes. Normalmente vale a pena pegar um tempo para ir mais afundo nos seus dados e ver todas as colunas com dados vazios)

Se você tem certeza que qur tirar os valores vazios, pandas possui uma função, `dropna()`, que ti ajudará com isso.

In [9]:
# removendo todas as linhas que possuem valores vazios
nfl.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


Meu deus !! Parece que removemos todos os dados !! Isso ocorreu por que todas as linhas do nossos dados tinha no minimo 1 valor vazio. Nós podemos ter alguma sorte removendo todas as colunas que tenham pelo menos 1 valor faltante:

In [10]:
colunas_com_valores_faltantes = nfl.dropna(axis=1)
colunas_com_valores_faltantes

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407683,2017-12-31,2017123101,29,4,1,0,-4,1,0,0,...,1,BAL,0,3,0,2,0,0.0,0.0,2017
407684,2017-12-31,2017123101,29,4,1,14,-4,1,0,0,...,0,None,2,2,0,2,0,0.0,0.0,2017
407685,2017-12-31,2017123101,29,4,1,14,9,1,13,0,...,0,None,2,2,0,2,0,0.0,0.0,2017
407686,2017-12-31,2017123101,30,4,1,10,-1,1,-1,0,...,0,None,0,2,0,2,0,0.0,0.0,2017


In [11]:
#Quantos valores perdemos?

print("Colunas originais no dataset: %d \n" %nfl.shape[1])
print("Colunas com valores vazios retiradaos: %d" %colunas_com_valores_faltantes.shape[1])

Colunas originais no dataset: 102 

Colunas com valores vazios retiradaos: 41


<h1>Preenchendo valores vazios automaticamente </h1>

Outra opção é tentar preencher os dados vazios. Vamos pegar um pequeno sub-set dos nossos dados:

In [12]:
subset_nfl = nfl.loc[:, "EPA":"Season"].head()
subset_nfl

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Podemos usar a função `fillna ()` do Panda para preencher os valores ausentes em um dataframe para nós. Uma opção que temos é especificar o que queremos que os valores NaN sejam substituídos. Aqui, estou dizendo que gostaria de substituir todos os valores NaN por 0.

In [14]:
subset_nfl.fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


Eu também poderia ser um pouco mais experiente e substituir os valores ausentes por qualquer valor que vier diretamente depois deles na mesma coluna. (Isso faz muito sentido para conjuntos de dados em que as observações têm algum tipo de ordem lógica.)

In [15]:
subset_nfl.fillna(method='bfill', axis=0).fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


<h1> Exercícios </h1>

# 1) Take a first look at the data

Rode a próxima célula para carregar os bibliotecas e o dataset que iremos utilizar:

In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv("Building_Permits.csv")

np.random.seed(0)

C:\Users\thiag\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Veja as primeiras 5 linhas de `df`:

In [17]:
#Seu Código aqui

<details><summary>Clique aqui para ver a resposta.</summary>

```python
df.head()
```

</details>

# 2) How many missing data points do we have?

Qual a % de valores perdidos? Sua resposta deve ser entre 0 e 100.

In [22]:
#Digite seu código abaixo:

#percent_missing = 


<details><summary>Clique aqui para ver a resposta.</summary>

```python
missing_values_count = df.isnull().sum()

total_celulas = np.product(df.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing/total_celulas)*100
```

</details>

# 3) Retirando valores vazios: Linhas

Se você remover todas as linhas de df que tenham valores vazios, quantas linhas sobram?

Não mude seu df original

In [24]:
#Digite seu código abaixo:

<details><summary>Clique aqui para ver a resposta.</summary>

```python
df.dropna()
```

</details>

# 4) Retirando valores vazios: Colunas

Agora tente remover todas as colunas com valores vazios. 
- Crie um novo DataFrame chamado `df_na_dropped` que contenha todas as colunas com valores vazios excluídos  
- Quantos colunas foram removidas dos dados originais? Use esses valores na variavél colunas_excluidas

In [27]:
#Digite seu código abaixo:

<details><summary>Clique aqui para ver a resposta.</summary>

```python
df_na_dropped= df.dropna(axis=1)

colunas_excluidas = df.shape[1] - df_na_dropped.shape[1]
```

</details>

# 5) Preenchendo valores vazios automaticamente

Tente substituir todos os NAN no DataFrame `df`  com valores que estão diretamente acima.  Coloque o resultado em outro DataFrame'

In [26]:
#Digite seu código abaixo:

<details><summary>Clique aqui para ver a resposta.</summary>

```python
df1 = df.fillna(method='bfill', axis=0).fillna(0)
```

</details>